## 1. Farmers' Market Expansion
<p>You have been hired by a farming organisation that helps local farmers sell their products. They want to know whether they should open up a new farmers' market to sell dairy products from nearby farmers. They have supplied you with daily shopping data from a panel of local households from 2019 to 2020. </p>
<p>The organization will make their decision based on whether dairy products are popular in the area, and whether sales are trending in a positive direction. To answer these questions, they want three pieces of data:</p>
<ol>
<li>What was the total number of purchases of dairy products for each month of 2020 (i.e., the <code>total_sales</code>)?</li>
<li>What was the total share of dairy products (out of all products purchased) for each month of 2020 (i.e., the <code>market_share</code>)?</li>
<li>For each month of 2020, what was the percentage increase or decrease in total monthly dairy purchases compared to the same month in 2019 (i.e., the <code>year_change</code>)?</li>
</ol>
<p>The organization handles not only dairy farmers, but also those with chicken farms. As a result, they are only interested in these three categories (which they treat as dairy): ‘whole milk’, 'yogurt' and 'domestic eggs'.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5960/datasets/463543c8c38957ca5b95d93b02f2cb1bec53334f/diagram.PNG" alt="Database Schema" width="400px"></p>

In [3]:
%%sql
postgresql:///groceries
    
WITH ms AS (
SELECT EXTRACT(MONTH FROM fulldate) AS month, COUNT(*) AS total_sales
FROM purchases_2020 AS p 
GROUP BY month),

total AS (
SELECT EXTRACT(MONTH FROM p.fulldate) AS month, COUNT(*) AS total_sales_diary_2020
FROM purchases_2020 AS p
LEFT JOIN categories AS c
ON p.purchaseid = c.purchase_id
WHERE category IN ('whole milk','yogurt', 'domestic eggs')
GROUP BY month),

chan AS (
SELECT p.month, COUNT(*) AS total_sales_diary_2019
FROM purchases_2019 AS p
LEFT JOIN categories AS c
ON p.purchase_id = c.purchase_id
WHERE category IN ('whole milk','yogurt', 'domestic eggs')
GROUP BY month
)


SELECT t.month, total_sales_diary_2020 AS total_sales,
ROUND(100*total_sales_diary_2020/total_sales::decimal,2) AS market_share,
ROUND(100*(total_sales_diary_2020 - chan.total_sales_diary_2019)/chan.total_sales_diary_2019::decimal,2) AS year_change
FROM total AS t
LEFT JOIN ms
ON t.month = ms.month
LEFT JOIN chan
ON t.month = chan.month
ORDER BY t.month;



12 rows affected.


month,total_sales,market_share,year_change
1.0,228,12.47,48.05
2.0,165,11.11,5.10
3.0,216,12.05,27.81
4.0,218,13.09,26.01
5.0,195,11.40,17.47
6.0,196,10.94,20.99
7.0,195,11.85,-0.51
8.0,231,11.77,32.00
9.0,180,11.16,4.65
10.0,193,11.61,17.68
